In [200]:
# coding: utf-8
import requests
from bs4 import BeautifulSoup
import pandas as pd

website_prefix = "https://www.reuters.com/finance/stocks/financial-highlights/"
companies_url = {"LVMH":"LVMH.PA","Airbus":"AIR.PA","Danone":"DANO.PA"}

def get_soup(url):
    res = requests.get(url)
    if res.status_code == 200:
        html_code = res.text
        soup = BeautifulSoup(html_code,"html.parser")
    return soup

def get_share_price(company):
    url = website_prefix + companies_url[company]
    soup = get_soup(url)
    specific_style = "font-size: 23px;"
    share_price = soup.find("span", style = specific_style).text
    return float(share_price.strip())

def get_pct_change(company):
    url = website_prefix + companies_url[company]
    soup = get_soup(url)
    specific_class = "valueContentPercent"
    pct_change = soup.find("span", class_ = specific_class).text
    return float(pct_change.strip()[1:-2]) # returns float x for a pct x%

def get_q4_sales(company):
    url = website_prefix + companies_url[company]
    soup = get_soup(url)
    specific_class = "stripe"
    html_code = soup.find("tr", class_ = specific_class).text
    q4_sales = html_code.split("\n")[3] # obtaining sales qty as string
    return float(q4_sales.replace(",","")) # transforming to float number

def get_shares_owned(company):
    url = website_prefix + companies_url[company]
    soup = get_soup(url)
    spc_class = "dataTable"
    spc_width = "100%"
    spc_cellpadding = "0"
    spc_cellspacing = "1"
    html_code = soup.find("table", class_ = spc_class, width = spc_width, \
                        cellpadding = spc_cellpadding, \
                        cellspacing = spc_cellspacing).text
    shares_owned = html_code.strip().split("\n")[1]
    return float(shares_owned[:-1]) # returns float x for a pct x%

def get_div_yield(company):
    url = website_prefix + companies_url[company]
    soup = get_soup(url)
    spc_class = "dataTable"
    spc_width = "100%"
    spc_cellpadding = "1"
    spc_cellspacing = "0"
    html_code = soup.findAll("table", class_ = spc_class, width = spc_width, \
                        cellpadding = spc_cellpadding, \
                        cellspacing = spc_cellspacing)[1].text
    # taking the 3 first div_yields as list of string
    div_yield_str = html_code.strip().split("\n")[6:9]
    # transforming list of string in a list of floats using map
    div_yield = list(map(lambda div: float(div), div_yield_str))
    return div_yield

def get_financials(company):
    financials = {}
    financials["SP (€)"] = get_share_price(company)
    financials["% change"] = get_pct_change(company)
    financials["Q4 sales (m€)"] = get_q4_sales(company)
    financials["% of shares owned by inst. inv."] = get_shares_owned(company)

    financials["DY Company(%)"], financials["DY Industry(%)"], \
    financials["DY Sector(%)"] = get_div_yield(company)
    return financials

all_financials = {}

for company in companies_url.keys():
    all_financials[company] = get_financials(company)

financials_df = pd.DataFrame(all_financials).transpose()
financials_df

,% change,% of shares owned by inst. inv.,DY Company(%),DY Industry(%),DY Sector(%),Q4 sales (m€),SP (€)
LVMH,-1.26,20.63,1.82,1.67,2.15,13667.7,261.95
Airbus,-1.68,43.60,1.66,1.48,1.47,23493.0,97.33
Danone,-2.14,50.60,2.90,2.78,2.48,6072.6,63.93


In [202]:
all_financials

{'LVMH': {'SP (€)': 261.95,
  '% change': -1.26,
  'Q4 sales (m€)': 13667.7,
  '% of shares owned by inst. inv.': 20.63,
  'DY Company(%)': 1.82,
  'DY Industry(%)': 1.67,
  'DY Sector(%)': 2.15},
 'Airbus': {'SP (€)': 97.33,
  '% change': -1.68,
  'Q4 sales (m€)': 23493.0,
  '% of shares owned by inst. inv.': 43.6,
  'DY Company(%)': 1.66,
  'DY Industry(%)': 1.48,
  'DY Sector(%)': 1.47},
 'Danone': {'SP (€)': 63.93,
  '% change': -2.14,
  'Q4 sales (m€)': 6072.6,
  '% of shares owned by inst. inv.': 50.6,
  'DY Company(%)': 2.9,
  'DY Industry(%)': 2.78,
  'DY Sector(%)': 2.48}}

In [203]:
pd.DataFrame(all_financials)

,LVMH,Airbus,Danone
% change,-1.26,-1.68,-2.14
% of shares owned by inst. inv.,20.63,43.60,50.60
DY Company(%),1.82,1.66,2.90
DY Industry(%),1.67,1.48,2.78
DY Sector(%),2.15,1.47,2.48
Q4 sales (m€),13667.70,23493.00,6072.60
SP (€),261.95,97.33,63.93
